In [11]:
import pandas as pd 
import urllib.request 
from bs4 import BeautifulSoup

We have already scraped the tables through the use of Selenium and Webdriver:

document.querySelectorAll('[data-id]')[0].children[1]

We have 111 documents that we need to scrape through and we are going to use BeautifulSoup to do that

In [17]:
# parse the code and this is one example .
soup = BeautifulSoup(open('1.txt'), 'html.parser')
print(soup)

<tbody><tr class="odd" role="row"><td class="sorting_1" tabindex="0"><a href="http://1stacesecurity.co.uk" target="_blank">1st ACE Security</a></td><td><div class="link-wrap"><a data-latitude="51.3983" data-longitude="0.0309737" data-trigger-location-change="true" href="#">Bromley, England</a>; <a data-latitude="51.1615" data-longitude="0.256178" data-trigger-location-change="true" href="#">Southborough, United Kingdom</a></div></td><td></td></tr><tr class="even" role="row"><td class="sorting_1" tabindex="0"><a href="http://1stchoiceoftexas.com" target="_blank">1st Choice Security &amp; Investigations</a></td><td><div class="link-wrap"><a data-latitude="30.4472" data-longitude="-97.6474" data-trigger-location-change="true" href="#">Pflugerville, Texas</a></div></td><td>Yes</td></tr><tr class="odd" role="row"><td class="sorting_1" tabindex="0"><a href="http://1stglobalintegrated.com/" target="_blank">1st Global Integrated Electronic Security Systems</a></td><td><div class="link-wrap"><a

In [13]:
import key as key
import googlemaps

# google maps api key
gmaps = googlemaps.Client(key=key.key['key'])

In [66]:
def parse_soup(soup):
    all_integrators = []
    for integrator in soup.find_all('tr'):
        hrefs = integrator.find_all('a')
        href = hrefs[0]
    #     grab the first. 
        longitude = hrefs[1]['data-longitude']
        latitude = hrefs[1]['data-latitude']

        lookup = gmaps.reverse_geocode((latitude, longitude))
        for component in lookup[0]['address_components']:
            if 'country' in component['types']:
                country = component['long_name']
        
        item = {
            'url': href['href'],
            'name': href.get_text(),
            'primary_loc': hrefs[1].get_text(),
            'country': country or ""
        }
        all_integrators.append(item)
    return all_integrators

In [51]:
# for parsing out the country in a google maps reverse geocode api call
lookup = gmaps.reverse_geocode(('14.5794', '121.036'))
for component in lookup[0]['address_components']:
    if 'country' in component['types']:
        print(component['long_name'])

Philippines


In [70]:
# Collect all the soups together from html files

soups = []

for x in range(1, 112): 
    try: 
        soups += BeautifulSoup(open('{0}.txt'.format(x)), 'html.parser')
    except:
        soups += BeautifulSoup(open('{0}'.format(x)), 'html.parser')    

In [74]:
# now that we have all these soups, we can create our array for ingestion into pandas 

integrators_total = []

for soup in soups:
    try: 
        integrators_total += parse_soup(soup)
    except: 
        pass

In [75]:
ints = pd.DataFrame(integrators_total)

In [77]:
# I like having pandas do the final export to csv ... 
ints.to_csv('integrators.csv')